In [97]:
from io import open as openio # used for parsing the SpamAssassin dataset, needed for 
import os
import re
import pandas as pd
from tqdm import tqdm

class Zerofucks:
    
    def __init__(self):
        self.df = pd.DataFrame(
            {'file': [], 'line_no': [], 'badword': [], 'content': []}, 
            columns=['file', 'line_no', 'badword', 'content']
        )
    
    def __makeRegex__(self, shitToFind=["fuck", "shit"]):
        '''
        shitToFind is a list of word literals,
        uses word boundary \b

        returns a regex string
        '''
        shitString = ''
        for word in shitToFind:
            # shitString += r"\b" + word + r"\b|"
            
            # case insensitive (?i) has deprecation warning:
            # https://github.com/bottlepy/bottle/issues/949
            shitString += r"(?i)\b" + word + r"\b|"
        return shitString[:-1]
        
    def write_df(self, path='./', filename='out.csv'):
        '''
        path: path to where you want to put the file
        filename: name of file
        
        Writes the csv from self.df to a csv file
        '''
        try:
            self.df.to_csv(path_or_buf=path + filename, index=False, chunksize=1000)
            print("File written to {}".format(path + filename))
        except:
            print("Unable to write csv, did you run .find_shit() yet?")
            
    def erase_shit(self, input_file, postfix='', bad_words=['fuck', 'shit']):
        '''
        Erases any bad words from the user specified files.
        
        Takes an input file and writes to an outputfile.
        ALSO returns a dictionary for forensic - what did you change - purposes
        
        input_file: full path to the file you want to process / remove bad words from
        postfix: optional string value you want to add to the end of the outputfile. If blank, it'll overwrite the input file.
        '''
        
        print('Replacing bad words in {}...'.format(input_file))
        try:
            #infile = open(input_file, mode='r').readlines()
            tempfile = open(input_file + '.tmp', mode='w')
            with open(input_file, 'r', encoding='iso-8859-15') as infile:
#                 outfile = open(output_file, mode='w')
                shit_dict = {}
                for content in tqdm(infile):
                    if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
        #                 print('Replacing {}...'.format(re.findall(self.__makeRegex__(bad_words), content)[0]))
                        if re.findall(self.__makeRegex__(bad_words), content)[0] in shit_dict:
                            shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] += 1
                        else:
                            shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] = 1
                    # the '' in this line is what to replace the bad words with. In this case, nothing at all.
                    tempfile.write(re.sub(self.__makeRegex__(bad_words), '', content))
            # write the tempfile contents to the original file  
            print('overwriting {} with {}'.format(input_file+'.tmp', input_file+postfix))
            os.rename(input_file + '.tmp', input_file+postfix)
            return shit_dict
        except ValueError:
            print(f'Unable to replace bad words in {input_file}.')
            
    def erase_shit_test(self, root_paths, postfix='', bad_words=['fuck', 'shit']):
        '''
        root_paths: LIST of directories to replace shit in. Must be a list. Cannot be a specific file.
        postfix: string to add to the end of the name of the output file. If input is 'myFile.csv' with a postfix of '_new', the scrubbed file would be 'myFile.csv_new'. No prefix overwrites the existing file.
        bad_words: optional list of words to replace in the files within root_paths
        '''
        for root_path in root_paths:
            for (root, dirs, files) in os.walk(root_path):
                 for name in files:
                    self.erase_shit(root+'/'+name, postfix, bad_words)
    
    def find_shit(self, root='./', bad_words=['fuck', 'shit'], include_content=True):
        '''
        root: the directory root where you want the crawl to start
        bad_words: list of words you want to search for.
            These can also be regular expressions. It will
            match partials, so 'fuck' will match 'fucking',
            'unfuckingbelievable', etc. Use \b and similar
            to constrain to word boundaries.
        
        The meat and potatoes, this is what conducts the walk
        and writes the resultant dataframe to self.df.
        
        To export the df to a csv, use the write_df() method.
        '''        
        for item in os.walk(root):
            
            # keep records for each directory parsed
            file_df = []
            line_no_df = []
            badword_df = []
            if include_content:
                content_df = []
            
            for file in item[2]:
                print('Searching {}'.format(item[0]+'/'+file))
                try:
                    openfile = open(item[0] + '/' + file, "r").readlines()
                    for line_no, content in enumerate(openfile):
                        if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
                            for i, badword in enumerate(re.findall(self.__makeRegex__(bad_words), content)):
                                file_df.append(item[0]+'/'+file)
                                line_no_df.append(line_no)
                                badword_df.append(re.findall(self.__makeRegex__(bad_words),content)[i])
                                # rstrip to remove newline character
                                if include_content:
                                    content_df.append(content.rstrip())
                except:
                    pass
                            
#             print(content_df)

            # write the records to the dataframe with each dir parsed
            if include_content:
                self.df = self.df.append(pd.DataFrame({'file': file_df, 
                                'line_no': line_no_df, 
                                'badword': badword_df,
                                'content': content_df},
                                columns=['file', 'line_no', 'badword', 'content']),
                                ignore_index=True)
            else:
                self.df = self.df.append(pd.DataFrame({'file': file_df, 
                    'line_no': line_no_df, 
                    'badword': badword_df},
                    columns=['file', 'line_no', 'badword']),
                    ignore_index=True)
        
        # change the silly auto-detected float line_no to an integer
        self.df['line_no'] = self.df.copy()['line_no'].apply(lambda x: int(x))
        # return self.df
        print("Dataframe successfully created, \n \
            use <obj>.df to print the df, or \n \
            .write_df() method to write to file.")

In [ ]:
# ERASE SHIT BACKUP
    def erase_shit(self, input_file, output_file, bad_words=['fuck', 'shit']):
        '''
        Erases any bad words from the user specified files.
        
        Takes an input file and writes to an outputfile.
        ALSO returns a dictionary for forensic - what did you change - purposes
        '''
        
        print('Replacing bad words in {}...'.format(input_file))
        try:
            #infile = open(input_file, mode='r').readlines()
            with open(input_file, 'r', encoding='iso-8859-15') as infile:
                outfile = open(output_file, mode='w')
                shit_dict = {}
                for content in tqdm(infile):
                    if len(re.findall(self.__makeRegex__(bad_words), content)) > 0:
        #                 print('Replacing {}...'.format(re.findall(self.__makeRegex__(bad_words), content)[0]))
                        if re.findall(self.__makeRegex__(bad_words), content)[0] in shit_dict:
                            shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] += 1
                        else:
                            shit_dict[re.findall(self.__makeRegex__(bad_words), content)[0]] = 1
                    outfile.write(re.sub(self.__makeRegex__(bad_words), ' ', content))
                return shit_dict
        except ValueError:
            print(f'Unable to replace bad words in {input_file}.')

Find words in files

Use this to find bad words in multiple files

In [98]:
# instantiate class
myFinder = Zerofucks()

# load in bad words list from file,
# write to list my_badwords
my_badwords = []
badwords_file = open('./bad_words.txt', "r").readlines()
for line in badwords_file:
    my_badwords.append(line.rstrip())

In [4]:
# find stuff!
myFinder.find_shit('./course_dump', my_badwords, include_content=False)

/home/greg/miniconda3/envs/utc/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/greg/miniconda3/envs/utc/lib/python3.6/site-packages/ipykernel_launcher.py:93: DeprecationWarning: Flags not at the start of the expression '(?i)\\ba55\\b|(?i)\\ban' (truncated)


Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/intro_to_neural_networks.ipynb
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/readme.md
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/assets/images/relu.png
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/assets/images/neuralnet.png
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/assets/datasets/titanic_test.csv
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/assets/datasets/titanic_train.csv
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/images/ann-perceptron-update.png
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/images/ann-perceptron-math.png
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/images/ann-layers.png
Searching ./course_dump/week12/day1_neural_networks/nnet-intro-sklearn/images/ann-perceptron.png
Searching ./course_

Searching ./course_dump/week10/day5_emr_and_hive/Hive-lab/AWS-Hive-solutioncode.ipynb
Searching ./course_dump/week10/day2_big_data_intro/readme.md
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/spark-practice-caltrain-starter.ipynb
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/spark-lab-EXTRA_solutions.ipynb
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/spark-practice-caltrain-solutions.ipynb
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/spark-lab-EXTRA_starter.ipynb
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/churn.csv
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/assets/images/upload.png
Searching ./course_dump/week10/day2_big_data_intro/spark_lab/data/201508_trip_data.csv
Searching ./course_dump/week10/day2_big_data_intro/intro_to_spark/readme.md
Searching ./course_dump/week10/day2_big_data_intro/intro_to_spark/intro-to-spark-solution-code.ipynb
Searching ./course_dump/week10/day2_big_data_int

Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0131.b715566061ab90d0f8745ce0ac010832
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0681.a0c682a61d47a58820ffa73d5555f3fb
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1149.68dbf497c1a8748757b6acc50fb74ed0
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1401.2d453d45aa06dec5a5a98bba8d81fdc5
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/2281.ce69a29fc23c4a9990ccc723e4da6898
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0517.a379a642533f85c04e63d9f3a0813b63
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0545.f3614ca8b5095f9b69b7a3ff336d5aac
Search

Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/2066.dce0718b87da2153138f0037c89fb38a
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0376.c0225fd19682f7ac58d090b6528af380
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1288.9235f568fa35fd2fcda2c067543fbd0e
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1778.5e3a4fdad399e2557d6921d7e938faef
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1275.7f5370e380b5cedb37bad38bb1ea1484
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1794.6057cf5e982869286b4742cee5639a4b
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/1284.bacf47285c3f6a514443b4d013d5736a
Search

...

Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0372.216f90ef52558ed24402e192586a40e8
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0005.1f42bb885de0ef7fc5cd09d34dc2ba54
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0170.fe4f77fa9456b48dffa9288074b2bb2a
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0188.6590e73ef71e79c5b6adedbacf91ac8c
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0189.3d89383221aa3fb155a099838ce9c40a
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0085.6e7b1a983ab05445a7eaffcbb6811d3f
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0392.9e194dfff92f7d9957171b04a8d4b957
Searching ./course_dump/week08/day

Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0311.fad7da9629598eaffeaf6896bdf32d9c
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0158.ff5dce5446d2ec91f0caffeffdd48852
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0219.0f66069db1b4e25ba851233ce4a107c4
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0207.3adcb1a14977a49cac8f6e10f64ac6f7
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0035.8e582263070076dfe6000411d9b13ce6
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0309.2a74113b0330ea76cecd28571fc6f7fe
Searching ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/spam/0419.a42a284750591b454968a76dfab38370
Searching ./course_dump/week08/day

Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Anaheim-max-50.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Bothell-max-10000.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Aurora-max-10000.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Mount Laurel-max-10000.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Cupertino-max-10000.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Duluth-max-100.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Venice-max-10000.csv
Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Bellevue.csv
Searching ./course_dump/week08/day5_more_p

Searching ./course_dump/week08/day5_more_pymc3/project-4-solution/data/indeed-scraped-job-postings-Scottsdale-max-10000.csv
Searching ./course_dump/week11/day2_svm_and_flask/flask_lesson/readme.md
Searching ./course_dump/week11/day2_svm_and_flask/flask_lesson/flask-lesson-solutions.ipynb
Searching ./course_dump/week11/day2_svm_and_flask/flask_lesson/flask-lesson.ipynb
Searching ./course_dump/week11/day2_svm_and_flask/flask_lesson/datasets/readme.md
Searching ./course_dump/week11/day2_svm_and_flask/flask_lesson/datasets/titanic.csv
Searching ./course_dump/week11/day2_svm_and_flask/support-vector-machines-lab-2/svm-lab-starter-code.ipynb
Searching ./course_dump/week11/day2_svm_and_flask/support-vector-machines-lab-2/solution-code/svm-lab-solution-code.ipynb
Searching ./course_dump/week11/day5_networks/tube_network/London_transport-starter-code.ipynb
Searching ./course_dump/week11/day5_networks/tube_network/solution-code/London_transport-solution-code.ipynb
Searching ./course_dump/week11/

Searching ./course_dump/week11/day1_sparkml_and_svm/support-vector-machines-lesson/code/svm_plotter.py
Searching ./course_dump/week11/day1_sparkml_and_svm/spark-ml-lesson/spark_ml_lesson.ipynb
Searching ./course_dump/week11/day1_sparkml_and_svm/spark-ml-lesson/readme.md
Searching ./course_dump/week11/day1_sparkml_and_svm/spark-ml-lesson/datasets/sample_linear_regression_data.txt
Searching ./course_dump/week11/day1_sparkml_and_svm/spark-ml-lesson/datasets/sample_libsvm_data.txt
Searching ./course_dump/week11/day1_sparkml_and_svm/spark-ml-lesson/datasets/sample_multiclass_classification_data.txt
Searching ./course_dump/week11/day1_sparkml_and_svm/support-vector-machines-lab-1/exploring-svms-sklearn-example.ipynb
Searching ./course_dump/week11/day1_sparkml_and_svm/support-vector-machines-lab-1/solution-code/exploring-svms-sklearn-example-solutions.ipynb
Searching ./course_dump/week09/day2_arima_models/4_timeseries_arima_lab/ARIMA-lab-startercode.ipynb
Searching ./course_dump/week09/day2_a

Write the no-output version to a file 

In [5]:
# write the no-output version to a file
myFinder.df[['file', 'badword', 'line_no']].head()

,file,badword,line_no
0,./course_dump/week12/day1_neural_networks/nnet...,Sex,1122
1,./course_dump/week12/day1_neural_networks/nnet...,Sex,0
2,./course_dump/week12/day1_neural_networks/nnet...,Israel,323
3,./course_dump/week12/day1_neural_networks/nnet...,Sex,0
4,./course_dump/week12/day1_neural_networks/nnet...,Butt,537


In [6]:
res = myFinder.df[['file', 'badword', 'line_no']]

In [7]:
res['badword'] = res['badword'].apply(lambda x: x.lower())

/home/greg/miniconda3/envs/utc/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
res.to_csv('./output_no_content.csv', index=False, chunksize=1000)

Just make a snippet of each offense, grouped by the file, with the top 5 offenses for each file

In [9]:
myFinder.df.groupby('file', as_index=False).head(5).to_csv('./head5.csv', index=False, chunksize=1000)

### Erase badwords from a single file and return a 'cleaned' file

Use this to erase badwords from a single file and return a cleaned file

In [3]:
# erase stuff! (returns a dictionary)
out_dict = myFinder.erase_shit('./course_dump/10.06-lesson-benfords-law/datasets/Austin_Finance_Online_eCheckbook.csv', './course_dump/10.06-lesson-benfords-law/datasets/out.csv', my_badwords)
# let's get a report of the stuff we replaced in the erase_shit() call:
out_df = pd.DataFrame.from_dict(out_dict, orient='index')
out_df.columns = ['count']
out_df.sort_values('count', ascending=False)[:10]

  0%|          | 34/118544 [00:00<05:54, 333.86it/s]

Replacing bad words in ./course_dump/10.06-lesson-benfords-law/datasets/Austin_Finance_Online_eCheckbook.csv...


100%|██████████| 118544/118544 [04:08<00:00, 477.37it/s]


,count
Drugs,172
Drug,100
DRUG,82
Dog,54
Crack,28
MUFF,20
GUN,18
COX,16
Spac,13
FLANGE,13


### Erase badwords from multiple files, in-place
Use this to erase an entire directory of shitty files full of shit

In [14]:
todelete = pd.read_csv('./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/todelete.csv')

In [16]:
todelete.head(2)

,folder,filename,file,badword
0,easy_ham,0094.b7bf14fae9c31d0516cfe00dd9ab068d,./course_dump/week08/day1_intro_to_bayesian_st...,10
1,easy_ham,0161.5703ef6933a7f189ab48da12a925f5e4,./course_dump/week08/day1_intro_to_bayesian_st...,3


In [19]:
# /home/greg/Documents/ga/profanityParser/course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab
!pwd

/home/greg/Documents/ga/profanityParser


In [24]:
for folder, file in zip(todelete['folder'][:2], todelete['filename'][:2]):
    path = './course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/' + folder + '/' + file
    print(path)
    !wc -l $path

./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0094.b7bf14fae9c31d0516cfe00dd9ab068d
309 ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0094.b7bf14fae9c31d0516cfe00dd9ab068d
./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0161.5703ef6933a7f189ab48da12a925f5e4
127 ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets/easy_ham/0161.5703ef6933a7f189ab48da12a925f5e4


In [48]:
# I need to merge this into the main class probabry

root_paths = [
    './course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham',
    './course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/hard_ham',
    './course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/spam'
]
for root_path in root_paths:
#     i = 0
    for (root, dirs, files) in os.walk(root_path):
        # prints files but not directories
        for name in files:
#             if i < 2:
            myFinder.erase_shit(root+'/'+name, root+'/'+'foo'+name, my_badwords)
#                 i += 1

74it [00:00, 994.48it/s]
88it [00:00, 1707.00it/s]
0it [00:00, ?it/s]

Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/foo0865.4b2110ced3bc59bb5684c5ecfa980dd1...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/1026.b2ee7b3cb90365641e465cfede58a672...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/0652.5ed46908836bacba742b337197ce3499...


188it [00:00, 1412.79it/s]
66it [00:00, 1176.78it/s]
104it [00:00, 1486.36it/s]
31it [00:00, 1226.79it/s]
0it [00:00, ?it/s]

Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/1508.334b2eb6c70ba66605ebefc97924e7b3...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/1595.fa533692bb0d635d6d1219663ee0f82c...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/foo2411.debe2dcb9f09b73d42f985bdd7913bb6...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/0109.4ccc46c546b93015aafdfc40495f187d...


75it [00:00, 1563.06it/s]
97it [00:00, 1323.56it/s]
100it [00:00, 1487.45it/s]
0it [00:00, ?it/s]

...

Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/2095.c0d4fbeea4942b5c55a59655694bdb9e...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/2133.d3d4f15eb898bffe360f99678d9cae2c...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/2291.3560a8bef18a21ce572bab2c364571d7...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/0294.145bbce5c541610ee520838637d2e112...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/foo0270.5a7d9b20e00fc24d6c929c198718f5a3...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/1425.7ed3569c34399b07b158eaead1c85cc8...


63it [00:00, 1530.00it/s]
94it [00:00, 1416.36it/s]
88it [00:00, 1498.14it/s]
0it [00:00, ?it/s]

Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/foo1415.92155ef4a4f97a787d636892dd1a0219...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/foo1276.833510ba5d9158c1172f5edf1277d41c...
Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_test/easy_ham/1022.73ab70b91862d709897cfe3dd5bb22a0...


391it [00:00, 1299.16it/s]


KeyboardInterrupt: 

In [95]:
myFinder.erase_shit_test(['./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_two/'], postfix='new_', bad_words=my_badwords)

117it [00:00, 1340.94it/s]

Replacing bad words in ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_two//0107.f1d4194b57840ea6587b9a73ed88e075...
overwriting ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_two//0107.f1d4194b57840ea6587b9a73ed88e075.tmp with ./course_dump/week08/day1_intro_to_bayesian_statistics/naive-bayes-spam-lab/datasets_two//0107.f1d4194b57840ea6587b9a73ed88e075new_
